In [3]:
#!/usr/bin/env python3

"""
- Script name: prepare_annotation_data
- Author: Dan Bright, cosmoid@tuta.io
- Description: A script to prepare spreadsheet data for annotation.
- Version 1.2
"""

import pandas as pd
import re, html
from pathlib import Path

In [ ]:
class CleanData:
    """
    Class that cleans and prepares the data training &

    Consumes:
        - input_data: list[tuple] = list of input data to clean, in form [(record_id:int, text:str)]
        - separate_slashes: bool = whether to separate slashes by a space [True|False]
        - remove_linebreaks: bool = whether to remove linebreaks & join by a space [True|False]
        - remove_non_alphanum: bool = whether to remove all non-alphanumeric characters [True|False]
        - ensure_encoding: bool = ensure all characters are correctly encoded (True|False)
    Produces:
        - list of cleaned data, in form [(record_id:int, text:str)]
    """

    def __new__(
        cls,
        input_data: list[tuple] = [],
        separate_slashes: bool = True,  # Default to True
        remove_linebreaks: bool = True,  # Default to True
        remove_non_alphanum: bool = True,  # Default to True
        ensure_encoding: bool = True,  # Default to True
    ):
        obj = super().__new__(cls)
        return obj._run_filters(
            docs=input_data,
            separate_slashes=separate_slashes,
            remove_linebreaks=remove_linebreaks,
            remove_non_alphanum=remove_non_alphanum,
            ensure_encoding=ensure_encoding,
        )

    def _run_filters(
        self,
        docs,
        separate_slashes,
        remove_linebreaks,
        remove_non_alphanum,
        ensure_encoding,
    ) -> list[tuple]:
        """
        Method to iterate the data & run filters.
        Returns: list of cleaned data in form [(record_id:int, text:str)]
        """
        filtered_docs: list[tuple] = []
        for record in docs:
            record_txt: str = record[1]
            record_txt = (
                self._separate_slashes(record_txt) if separate_slashes else record_txt
            )
            record_txt = (
                self._remove_linebreaks(record_txt) if remove_linebreaks else record_txt
            )
            record_txt = (
                self._remove_non_alphanum(record_txt)
                if remove_non_alphanum
                else record_txt
            )
            record_txt = (
                self._ensure_encoding(record_txt) if ensure_encoding else record_txt
            )
            filtered_docs.append((record[0], record_txt))
        return filtered_docs

    @staticmethod
    def _ensure_encoding(input: str) -> str:
        """
        Method to ensure characters are encoded correctly
        (i.e., no html entities, etc)
        """
        return html.unescape(input)

    @staticmethod
    def _separate_slashes(input: str) -> str:
        """
        Method to ensure all slashes within strings are surrounded by
        whitespace.
        """
        return re.sub(r"(?<!\s)/(?!\s)", " / ", input)

    @staticmethod
    def _remove_linebreaks(input: str) -> str:
        """
        Method to remove paragraphs breaks.
        """
        return " ".join(input.splitlines())

    @staticmethod
    def _remove_non_alphanum(input: str) -> str:
        """
        Method to remove all non-alphanumeric characters, except:
          - whitespaces
          - dots
          - forward slashes
        """
        return re.sub(r"\s+", " ", re.sub(r"[^\w\s\.\/]+", "", input))

In [11]:
class PrepareSample:
    """
    Class to prepare a random sample of a given column (`target_header`) of spreadsheet data
    for annotation.

    Any row of the input spreadsheet where any value is missing from the defined `required_columns`
    is dropped.

    Note: To perform extra processing, simply add new methods following the pattern of
    `_separate_slashes()`, and add the method to `__init__()`.

    Takes:
        - data_file: str = URL of the input data file.
        - sample_output_xlsx: str = URL of output .xlsx file containing sample data.
        - output_data_dir: str = URL of the txt output directory.
        - exclude_prev_sample: str = URL of xlsx file containing previously sampled files (if any)
        - exclude_id_header_map: tuple = maps headers of columns in exclude_sample and data_file; same values excluded in these
        - sample_size: int = Size of the random sample (number of spreadsheet rows).
        - target_header: str = Spreadsheet header of the column from which to extract data
          for writing to output text (.txt) files.
        - index_column_label: str = Name of the column in input spreadsheet containing row indices.
        - required_columns: dict[str:str] = Dictionary of columns to be retained in the
          output .xlsx file, in form {"original column name": "output column name"}.
        - write_xlsx: bool = Defines whether to write an output excel file (.xlsx). True|False.
        - write_txt: bool = Defines whether to write & save text files for each row's
          target_header column.
    Provides:
        - Method to ensure all slashes in a text string are surrounded by whitespace.
        - One text file (.txt) per randomly selected spreadsheet row, containing the
          appropriately formatted text extracted from the specified spreadsheet column (target_header),
          to be saved in the output directory.
        - One excel file (.xlsx) containing the randomly selected spreadsheet rows, with the only the
          require_columns included.
    Returns:
        - None.
    """

    def __init__(
        self,
        data_file: str,
        sample_output_xlsx: str,
        output_data_dir: str,
        exclude_prev_sample: str,
        exclude_id_header_map: tuple,
        sample_size: int,
        target_header: str,
        index_column_label: str,
        required_columns: dict[str:str],
        write_xlsx: bool,
        write_txt_files: bool,
    ) -> None:
        self._data_file: Path = Path(data_file).resolve(strict=True)
        self._sample_output_xlsx: str = sample_output_xlsx
        self._output_data_dir: Path = Path(output_data_dir).resolve(strict=False)
        self._exclude_prev_sample: Path = Path(exclude_prev_sample).resolve(
            strict=False
        )
        self._exclude_id_header_map: tuple = exclude_id_header_map
        self._sample_size: int = sample_size
        self._target_header: str = target_header
        self._required_columns: dict[str:str] = required_columns
        self._index_column_label: str = index_column_label
        self._random_sample: list[tuple] = []
        self._df_sel: pd.DataFrame = None
        self._df_sample: pd.DataFrame = None
        self._get_data()
        self._get_random_sample()
        if self._random_sample:
            # ADD PROCESSING METHODS _BELOW_ THIS LINE
            self._random_sample = CleanData(self._random_sample)
            # _DO_NOT_ ADD ANYTHING _BELOW_ THIS LINE
            self._write_to_xlsx() if write_xlsx else None
            self._to_txt_files() if write_txt_files else None

    def _get_data(self) -> None:
        """
        Method to extract cells from excel spreadsheet column.
          - Only extracts columns passed into class as `required_columns`.
          - Drops rows where any value is missing from `required_columns`.
        """
        pd.options.mode.use_inf_as_na = True
        df: pd.DataFrame = pd.read_excel(self._data_file)
        self._df_sel = df[[c for c in self._required_columns]].copy()
        self._df_sel.dropna(axis=0, inplace=True)
        print(f"{len(self._df_sel.index)} valid records have been initially extracted")

    def _get_random_sample(self) -> None:
        """
        Method to return random sample from a list of strings, as [(index (starting @ 1)), string)].
        """
        if self._exclude_prev_sample:
            df_prev: pd.DataFrame = pd.read_excel(self._exclude_prev_sample)
            values_to_exclude: list = df_prev[self._exclude_id_header_map[0]].to_list()
            self._df_sel = self._df_sel[
                self._df_sel[self._exclude_id_header_map[1]].isin(values_to_exclude)
                == False
            ]
        try:
            self._df_sample = self._df_sel.sample(n=self._sample_size)
            for idx in range(self._sample_size):
                self._random_sample.append(
                    (
                        self._df_sample.iloc[idx][self._index_column_label].astype(int),
                        self._df_sample.iloc[idx][self._target_header],
                    )
                )
        except ValueError as e:
            print(
                f"""
                Operation terminated; a sample could not be obtained.

                Looks like there were not enough valid records to create a 
                sample of the required size.
                
                Additional info: '{str(e)}.'
                """
            )

    def _write_to_xlsx(self):
        """
        Method to write the required columns from the sample to a .xlsx file.
        """
        # ensure index column is type INT
        self._df_sample[self._index_column_label] = self._df_sample[
            self._index_column_label
        ].astype(int)
        # sort by index column
        self._df_sample.sort_values(by=self._index_column_label, inplace=True)
        # rename columns if required
        self._df_sample.rename(columns=self._required_columns, inplace=True)
        # write to excel
        self._df_sample.to_excel(self._sample_output_xlsx, index=False)

    def _to_txt_files(self) -> None:
        """
        Method to write each list entry to a .txt file.
        """
        for record in self._random_sample:
            with open(self._output_data_dir / f"record_{record[0]}.txt", "w") as file:
                file.write(record[1])
        print(f"Text files written in {self._output_data_dir}. Job done.")

In [ ]:
"""define paths & parameters"""

# location of input data xlsx
DATA_FILE: str = "../../data/input/WIP_RP_VERSION_3a.xlsx"
# where to save output sample xlsx (note: saved in input dir, as still input data (sample of))
SAMPLE_OUTPUT_XLSX: str = "../../data/input/test_sample_1.xlsx"
# where to save output txt files
OUTPUT_DATA_DIR: str = "../../data/sample/test/txt"
# loc of xlsx of prev sampled rcds
EXCLUDE_PREV_SAMPLE: str = "../../data/input/test_sample_1.xlsx"
# column header name of values to exclude -
# mapped as (header in prev sampled (`EXCLUDE_PREV_SAMPLE`), header in input data `DATA_FILE`)
EXCLUDE_ID_HEADER_MAP: tuple = (
    "RECORD_ID",
    "RecNum",
)
# name of header of column to extract values from
TARGET_HEADER: str = "CLEANED Summary"
# name of header of the column to be regarded as record index or ID
INDEX_COLUMN_LABEL: str = "RecNum"
# whether to write the selected sample to a new output excel file
WRITE_XLSX: bool = True
# whether to write the values from selected TARGET_HEADER column to text files
WRITE_TXT_FILES: bool = True
# columns required to be imported -
#   in form {original col name: output col name for output xlsx file (can be same)}
REQUIRED_COLUMNS: list[str] = {
    INDEX_COLUMN_LABEL: "RECORD_ID",
    TARGET_HEADER: "SUMMARY",
}
# size of sample
SAMPLE_SIZE: int = 25

"""extract & prepare sample of report data for annotation"""

PrepareSample(
    data_file=DATA_FILE,
    sample_output_xlsx=SAMPLE_OUTPUT_XLSX,
    output_data_dir=OUTPUT_DATA_DIR,
    exclude_prev_sample=EXCLUDE_PREV_SAMPLE,
    exclude_id_header_map=EXCLUDE_ID_HEADER_MAP,
    sample_size=SAMPLE_SIZE,
    target_header=TARGET_HEADER,
    index_column_label=INDEX_COLUMN_LABEL,
    required_columns=REQUIRED_COLUMNS,
    write_xlsx=WRITE_XLSX,
    write_txt_files=WRITE_TXT_FILES,
)